In [1]:
import pandas as pd

# 0.0 Carga de datos

# MODIFICAR A CSVs MAS RECIENTE

In [55]:
# Este es el dataframe con los viajes registrados por mes(es)

df = pd.read_csv('ecobicis-2018-03.csv')

/Users/franciscopatino/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
# Este es el dataframe con los numeros de estación mas su geolocalización

# estaciones = pd.read_csv('stations.csv')

distancias = pd.read_csv('0_0_distancia_estaciones_esvelta.csv')

## 0.2 Limpia de datos

In [57]:
# No todos los ID de bici son String

df = df.astype({'Bici': str})

In [58]:
# Por el momento es mas facil dar de baja estaciones fuera del listado de estaciones

df = df.drop(df[df.Ciclo_Estacion_Retiro == 1001].index)
df = df.drop(df[df.Ciclo_Estacion_Retiro == 3000].index)
df = df.drop(df[df.Ciclo_Estacion_Arribo == 1001].index)
df = df.drop(df[df.Ciclo_Estacion_Arribo == 3000].index)

# 1.0 Agregar Datetime a Dataframe

In [59]:
import re
import datetime
from datetime import datetime

# Esta función usa búsqueda de Regex del texto para las columnas de fecha y hora
# Consigue los números de ambos, los une y usa la función de datetime para crear la
# fecha y hora. Esto se va a aplicar para crear datetime de retiro y arribo para 
# poder crear funciones mas complejas.

def fn_fecha(string_date, string_time):
    day = re.search('\d+', string_date)
    day_num = int(day.group(0))

    month = re.search('(?<=\/)..', string_date)
    month_num = int(month.group(0))

    year = re.search('(?<=\/..\/)....', string_date)
    year_num = int(year.group(0))

    hour = re.search('\d+', string_time)
    hour_num = int(hour.group(0))
    minute = re.findall('(?<=:)\d+', string_time)
    minute_num = int(minute[0])
    second_num = int(minute[1])

    try:
        date_and_time = datetime(year_num, month_num, day_num, hour_num, minute_num, second_num)
    except: 
        print('Error en formato de fecha!')
        date_and_time = 'Error!'
    
    return date_and_time

In [60]:
# Prueba de función:

string1 = df.Fecha_Retiro[0]
string2 = df.Hora_Retiro[737224]

fn_fecha(string1, string2)

datetime.datetime(2018, 3, 1, 23, 59, 57)

# 1.1 Agregar distancia de viajes

In [61]:
# Esta funcion regresa la distancia de viaje, despues de
# filtrar el dataframe por estación de retiro y arribo indicado

# Version compatible con dataframe esvelto de estaciones (sin importar orden de estaciones)

def fn_dist_ruta(df_distancias, est_retiro, est_arribo):
    try:
        distancia = df_distancias.loc[(df_distancias['est_retiro'] == est_retiro) 
                                                   & (df_distancias['est_arribo'] == est_arribo), 'distancia']
    except: 
        distancia = df_distancias.loc[(df_distancias['est_retiro'] == est_arribo) 
                                               & (df_distancias['est_arribo'] == est_retiro), 'distancia']
    valor = distancia.values[0]

    return valor

# Esta función crea una nueva columna en el dataframe de viajes
# que es la distancia entre las dos estaciones

def fn_distancias_viajes(df_viajes, df_distancias):
    df_viajes['distancia_viaje'] = df_viajes.apply(lambda x: fn_dist_ruta(df_distancias, 
                                                    x['Ciclo_Estacion_Retiro'], 
                                                    x['Ciclo_Estacion_Arribo']), axis =  1)

In [62]:
# Prueba de función

fn_dist_ruta(distancias, 1, 2)

0.4744363478885178

In [63]:
# Prueba de función

df_prueba_viajes = {'Ciclo_Estacion_Retiro': [1,2,3,4,5],
                    'Ciclo_Estacion_Arribo': [5,4,3,2,1]}
                
df_prueba_distancias = {'retiro_arribo': [[1, 5], [2,4], [3,3], [4,2], [5,1]], 
                        'distancia': [5.1, 7.2, 0, 7.2, 5.1], 
                        'est_retiro': [1, 2, 3, 4, 5], 
                        'est_arribo': [5, 4, 3, 2, 1]}

df_test_0_0 = pd.DataFrame(df_prueba_viajes)

df_test_0_1 = pd.DataFrame(df_prueba_distancias)

fn_distancias_viajes(df_test_0_0, df_test_0_1)

df_test_0_0.head()

,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,distancia_viaje
0,1,5,5.1
1,2,4,7.2
2,3,3,0.0
3,4,2,7.2
4,5,1,5.1


# 1.2 Velocidad de viaje

In [64]:
# Función para crear columnas de: 1) fecha arribo, 2) retiro, y 3) tiempo en uso

# Este paquete genera barras de progreso. No es esencial pero ayuda con ver progreso
# al procesar datasets muy grandes

from tqdm import tqdm

# Aplica la función que convierte el texto en fecha en una columna nueva

def fn_datetime_y_uso_df(dataframe):
    
    dataframe['Fecha_Retiro_datetime'] = dataframe.apply(lambda x: fn_fecha(x['Fecha_Retiro'], x['Hora_Retiro']), axis =  1)
    dataframe['Fecha_Arribo_datetime'] = dataframe.apply(lambda x: fn_fecha(x['Fecha_Arribo'], x['Hora_Arribo']), axis =  1)
    dataframe['tiempo_en_uso'] = dataframe.apply(lambda x: (x['Fecha_Arribo_datetime'] - x['Fecha_Retiro_datetime']), axis =  1)

# La función de velocidad intenta dividir distancia sobre tiempo con IF/ELSE y TRY/EXCEPT:
# tiene excepciones por la existencia de viajes de/hacia la misma estación
# y algunos peculiares donde el tiempo es 0.
    
def fn_velocidad(dataframe):
    velocidades = []
    for i in tqdm(range(len(dataframe))):
        index = dataframe.index[i]
        row = dataframe.loc[index]
        if row.distancia_viaje >= 0.0 and row.tiempo_en_uso.total_seconds() > 0.0:
            try: 
                velocidad = row.distancia_viaje / (row.tiempo_en_uso.total_seconds()/3600)
            except:
                velocidad = 0
        else:
            velocidad = 0
        velocidades.append(velocidad)
    dataframe['velocidad'] = velocidades

In [65]:
# Prueba de función

df_prueba_uso = {'Fecha_Retiro': ['01/03/2018', '02/03/2018', '03/03/2018'],
                    'Hora_Retiro': ['01:59:57','02:59:57','03:59:57'],
                    'Fecha_Arribo': ['01/03/2018', '02/03/2018', '03/03/2018'],
                    'Hora_Arribo': ['02:01:00','03:02:00','04:03:00']}
                
df_test_0_2 = pd.DataFrame(df_prueba_uso)

fn_datetime_y_uso_df(df_test_0_2)

df_test_0_2.head()

,Fecha_Retiro,Hora_Retiro,Fecha_Arribo,Hora_Arribo,Fecha_Retiro_datetime,Fecha_Arribo_datetime,tiempo_en_uso
0,01/03/2018,01:59:57,01/03/2018,02:01:00,2018-03-01 01:59:57,2018-03-01 02:01:00,00:01:03
1,02/03/2018,02:59:57,02/03/2018,03:02:00,2018-03-02 02:59:57,2018-03-02 03:02:00,00:02:03
2,03/03/2018,03:59:57,03/03/2018,04:03:00,2018-03-03 03:59:57,2018-03-03 04:03:00,00:03:03


# 2.0 Correr funciónes y exportar

In [46]:
fn_distancias_viajes(df, distancias)

In [ ]:
df.head()

In [48]:
fn_datetime_y_uso_df(df)

In [49]:
df.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,distancia_viaje,Fecha_Retiro_datetime,Fecha_Arribo_datetime,tiempo_en_uso
0,M,49,3307,365,01/03/2018,0:00:10,169,01/03/2018,0:13:05,2.729608,2018-03-01 00:00:10,2018-03-01 00:13:05,00:12:55
1,F,29,9218,137,01/03/2018,0:00:13,308,01/03/2018,0:18:34,3.561729,2018-03-01 00:00:13,2018-03-01 00:18:34,00:18:21
2,F,30,3945,54,01/03/2018,0:00:17,191,01/03/2018,0:12:13,2.019986,2018-03-01 00:00:17,2018-03-01 00:12:13,00:11:56
3,M,25,9971,130,01/03/2018,0:00:47,271,01/03/2018,0:12:40,2.551229,2018-03-01 00:00:47,2018-03-01 00:12:40,00:11:53
4,M,20,10551,415,01/03/2018,0:00:56,433,01/03/2018,0:05:18,1.037867,2018-03-01 00:00:56,2018-03-01 00:05:18,00:04:22


In [50]:
fn_velocidad(df)

100%|██████████| 737214/737214 [04:23<00:00, 2798.93it/s]


In [51]:
df.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,distancia_viaje,Fecha_Retiro_datetime,Fecha_Arribo_datetime,tiempo_en_uso,velocidad
0,M,49,3307,365,01/03/2018,0:00:10,169,01/03/2018,0:13:05,2.729608,2018-03-01 00:00:10,2018-03-01 00:13:05,00:12:55,12.679471
1,F,29,9218,137,01/03/2018,0:00:13,308,01/03/2018,0:18:34,3.561729,2018-03-01 00:00:13,2018-03-01 00:18:34,00:18:21,11.645980
2,F,30,3945,54,01/03/2018,0:00:17,191,01/03/2018,0:12:13,2.019986,2018-03-01 00:00:17,2018-03-01 00:12:13,00:11:56,10.156356
3,M,25,9971,130,01/03/2018,0:00:47,271,01/03/2018,0:12:40,2.551229,2018-03-01 00:00:47,2018-03-01 00:12:40,00:11:53,12.881380
4,M,20,10551,415,01/03/2018,0:00:56,433,01/03/2018,0:05:18,1.037867,2018-03-01 00:00:56,2018-03-01 00:05:18,00:04:22,14.260767


In [53]:
df.to_csv('0_2_df_viajes_marzo.csv', index=False)

In [66]:
distancias.head()

,retiro_arribo,distancia,est_retiro,est_arribo
0,"(1, 1)",0.000000,1,1
1,"(1, 2)",0.474436,1,2
2,"(1, 3)",0.981942,1,3
3,"(1, 4)",0.714133,1,4
4,"(1, 5)",0.458207,1,5
